In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 1239.13630853,  1235.28897801],
        [ 1210.58791069,  1119.62262721],
        [ 1488.54358671,  1496.84720701],
        [ 1235.50714814,  1374.72644697],
        [ 1513.5052299 ,  1532.86574706],
        [ 1321.96692402,  1406.57684477],
        [ 1260.74148969,  1268.48947433],
        [ 1281.27145791,  1297.22226973],
        [ 1745.92146421,  1703.80034601],
        [ 1683.12747108,  1667.66511186],
        [ 1506.42369744,  1583.20507441],
        [ 1190.37933729,  1105.12745532],
        [ 1292.02020419,  1261.14577118],
        [ 1342.59385687,  1253.89548693],
        [ 1321.78492207,  1251.18261254],
        [  975.6725308 ,   993.07325234],
        [ 1250.6564891 ,  1275.03109722],
        [ 1244.04283596,  1259.33838887],
        [ 1561.67136149,  1497.68151885],
        [ 1499.69155676,  1513.42899037],
        [ 1106.72374652,  1138.42906635],
        [ 1473.97520853,  1497.22406251],
        [ 1246.82819544,  1262.06300225],
        [ 1221.26562755,  1227.153

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 2e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 26
 AA' NZ     : 3.250e+02
 Factor NZ  : 3.510e+02
 Factor Ops : 6.201e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.92219433e+08  0.00000000e+00  1.13e+04 3.04e+03  1.00e+06     0s
   1  -4.45698866e+07 -3.52155865e+07  6.42e+02 1.73e+02  1.50e+05     0s
   2  -3.41874076e+07 -3.48404651e+07  6.16e-01 1.66e-01  1.19e+04     0s
   3  -3.42539420e+07 -3.43166850e+07  2.16e-02 5.83e-03  1.1

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[3.9148304033059235e-12,
 464.3045145248558,
 4.2971937653681865e-12,
 1361.720266062351,
 673.0431532966536,
 0.23191596000482306,
 1.296759928965704e-09,
 1.9855362090715883e-05,
 0.006276985856275855,
 1298.4768276643445,
 0.006279205801012905,
 0.006275945365614592,
 0.006120868956509665,
 113.26529538793416,
 1522.4489264816925,
 53.26000467524697,
 92.38773852665763,
 142.00341887178746,
 896.8546185329589,
 0.2275297143122792,
 0.09433628486138934,
 796.0349882040925,
 664.4721974134491,
 0.07995319918001287,
 87.1192623016824,
 1102.396463990994,
 0.09047428026891069,
 1223.8196157469351,
 0.441589401578905,
 1097.4440313553894,
 0.09457406814795788,
 0.0330013917892899,
 0.3288205679101328,
 238.63181515778527,
 1240.365801149077,
 7.136423188728147e-07,
 7.136311207632094e-07,
 7.136469474444513e-07,
 1041.3167897640444,
 0.7419846759731148,
 382.4237652074757,
 0.06287215238600607,
 0.0029367565588591556,
 0.0027779156895450866,
 0.003004337230178348,
 21.251302044359385,
 9